## Dependencies install
* Haiku

In [1]:
!pip install dm-haiku

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.3/374.3 kB 17.7 MB/s eta 0:00:00


## Helper functions
* Image Visualizer

In [4]:
import matplotlib.pyplot as plt

def visualize_image(image):
    plt.imshow(image)
    plt.axis("off")
    plt.show()

## Package import
* jax, jax.numpy
* haiku
* optax
* chex (to do)
* tqdm

In [5]:
import jax
import jax.numpy as jnp
import haiku as hk
import optax
key42 = jax.random.PRNGKey(42)

import random
random.seed(42)

from tqdm.notebook import tqdm

jax.devices()

ERROR:2026-01-30 05:09:26,933:jax._src.xla_bridge:487: Jax plugin configuration error: Exception when calling jax_plugins.xla_cuda12.initialize()
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/jax/_src/xla_bridge.py", line 485, in discover_pjrt_plugins
    plugin_module.initialize()
  File "/usr/local/lib/python3.12/dist-packages/jax_plugins/xla_cuda12/__init__.py", line 328, in initialize
    _check_cuda_versions(raise_on_first_error=True)
  File "/usr/local/lib/python3.12/dist-packages/jax_plugins/xla_cuda12/__init__.py", line 285, in _check_cuda_versions
    local_device_count = cuda_versions.cuda_device_count()
                         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
RuntimeError: jaxlib/cuda/versions_helpers.cc:113: operation cuInit(0) failed: Unknown CUDA error 303; cuGetErrorName failed. This probably means that JAX was unable to load the CUDA libraries.


[CpuDevice(id=0)]

## Generator

## Discriminator

## Data loading (Cifar-10)

In [9]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

def get_cifar10(batch_size):
    (x_train, _), (x_test, _) = tf.keras.datasets.cifar10.load_data()
    
    # Scale to [-1, 1]
    x_train = (x_train.astype(np.float32) - 127.5) / 127.5
    
    # Create a tf.data.Dataset from the numpy array
    ds = tf.data.Dataset.from_tensor_slices(x_train)
    ds = ds.shuffle(50000).batch(batch_size, drop_remainder=True)
    ds = ds.repeat().prefetch(tf.data.AUTOTUNE)
    # Convert to a numpy iterator for JAX
    return iter(tfds.as_numpy(ds))

In [15]:
image_array = next(get_cifar10(32))
image_array.shape

(32, 32, 32, 3)

## Data loading (CelebA)

In [9]:
import os
from PIL import Image
import numpy as np

class CelebALoader:
    def __init__(self, batch_size = 32, shuffle=False):
        dataset_path = "/kaggle/input/celeba-dataset"
        self.images_dir_path = os.path.join(dataset_path, "img_align_celeba/img_align_celeba")

        self.batch_size = batch_size
        
        self.images_filenames = os.listdir(self.images_dir_path)
        if shuffle:
            random.shuffle(self.images_filenames)
            
    def __iter__(self):
        batch = jnp.array([])
        for i, image_filename in enumerate(self.images_filenames):
            image_path = os.path.join(self.images_dir_path, image_filename)
            
            pil_image = Image.open(image_path)
            jnp_image = jnp.array(np.array(pil_image))
            jnp_image = jnp.expand_dims(jnp_image, 0)
            
            if i % self.batch_size == 0:
                batch = jnp_image
            else:
                batch = jnp.concatenate((batch, jnp_image), axis=0)
            
            if i % self.batch_size == self.batch_size - 1:
                yield batch
                batch = jnp.array([])
            
    def __len__(self):
        return len(self.images_filenames)

celeba_loader = CelebALoader(batch_size=32, shuffle=True)

# Check the dimension of the images
for i, batch in enumerate(celeba_loader):
    print(batch.shape)
    break

(32, 218, 178, 3)


## Loss function

## Training Loop

## Generation